In [1]:
import pandas as pd 
from os.path import join
from glob import glob
import joblib

import sys
sys.path.append('/home/monte.flora/python_packages/master/ml_workflow')
sys.path.append('/home/monte.flora/python_packages/scikit-explain')

import skexplain

In [2]:
for hazard in ['severe_wind', 'severe_hail']:
    # Load the WoFS-ML-Severe Models
    base_path = '/work/mflora/ML_DATA/MODEL_SAVES'
    model_name = 'LogisticRegression'

    model_paths = glob(join(base_path, f'{model_name}_first_hour_{hazard}*'))
    model_path = [m for m in model_paths if 'manual' not in m][0]

    model_data = joblib.load(model_path)
    
    # Load the WoFS data. 
    base_path = '/work/mflora/ML_DATA/DATA'
    data_path = join(base_path, f'original_first_hour_training_matched_to_{hazard}_0km_data.feather')
    df = pd.read_feather(data_path)

    model = model_data['model']
    features = model_data['features']

    X = df[features].astype(float)
    y = df[f'matched_to_{hazard}_0km'].astype(float)
    
    explainer = skexplain.ExplainToolkit(estimators=('LogisticRegression', model), X=X, y=y)
    results = explainer.pd(features='all',
             n_bootstrap=10,
             subsample=0.25,
             n_jobs=30, 
             n_bins=20)
    
    base_path = '/work/mflora/ML_DATA/'
    pd_fname = join(base_path,'PD_RESULTS', f'pd_results_all_models_{hazard}_first_hour.nc')

    explainer.save(pd_fname, data=results)

  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/113 [00:00<?, ?it/s]

In [2]:
import sys
sys.path.append('/home/monte.flora/python_packages/master/ml_workflow')
sys.path.append('/work/mflora/ROAD_SURFACE')

from probsr_config import PREDICTOR_COLUMNS,  TARGET_COLUMN
from calibration_classifier import CalibratedClassifier

name = 'Random Forest'
base_path = '/work/mflora/ROAD_SURFACE'
model = joblib.load(join(base_path, 'JTTI_ProbSR_RandomForest.pkl'))
calibrator = joblib.load(join(base_path, 'JTTI_ProbSR_RandomForest_Isotonic.pkl'))
model = CalibratedClassifier(model, calibrator)

df = pd.read_csv(join(base_path, 'probsr_training_data.csv'))
        
X = df[PREDICTOR_COLUMNS].astype(float)
y = df[TARGET_COLUMN].astype(float) 

In [ ]:
explainer = skexplain.ExplainToolkit(estimators=(name, model), X=X, y=y)
results = explainer.pd(features='all',
             n_bootstrap=10,
             subsample=0.25,
             n_jobs=30, 
             n_bins=20)

base_path = '/work/mflora/ROAD_SURFACE'
pd_fname = join(base_path,'pd_results', f'pd_rf_original.nc')

explainer.save(pd_fname, data=results)

  0%|          | 0/30 [00:00<?, ?it/s]